In [44]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
from sklearn.decomposition import PCA

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
MODEL_NAME = '../meta-llama/Llama-2-7b-chat-hf'

# Load our model from local
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME+'_model', device_map="auto", torch_dtype=torch.float16)

# And its associated tokenizer from local
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME+'_tokenizer')


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [34]:
ENABLE_SAMPLING_OUTPUT = True
ENABLE_NAIVE_OUTPUT = False
ENABLE_LM_HEAD_ACTIVATION = False
ENABLE_MODEL_LAYERS_ACTIVATION = True
ENABLE_MODEL_LAYERS_INDEX = 31
ENABLE_MODEL_LAYERS_NAME = 'mlp'

In [9]:
def get_activation(layer):
    def hook(module, input, output):
        layer.activation = output.detach()  # we use `detach` to separate this value from the computation graph
    return hook

In [12]:
if ENABLE_LM_HEAD_ACTIVATION:
    hook_ref = model.lm_head.register_forward_hook(get_activation(model.lm_head))
    input = tokenizer("This is a test sentence", return_tensors="pt").to(device)
    outputs = model(**input)  # Process the input
    activation = model.lm_head.activation  # Access the activation of the lm_head layer you just hooked
    hook_ref.remove()

In [23]:
if ENABLE_LM_HEAD_ACTIVATION:
    print(activation.shape)
    print(activation)

In [57]:
if ENABLE_MODEL_LAYERS_ACTIVATION and ENABLE_MODEL_LAYERS_NAME == 'mlp':
    hook_ref = model.model.layers[ENABLE_MODEL_LAYERS_INDEX].mlp.register_forward_hook(get_activation(model.model.layers[ENABLE_MODEL_LAYERS_INDEX].mlp))
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(["HOW ARE YOU", "HOW OLD ARE YOU"], return_tensors="pt", padding='longest').to(device)
    outputs = model(**inputs)
    activation = model.model.layers[ENABLE_MODEL_LAYERS_INDEX].mlp.activation
    hook_ref.remove()

In [53]:
if ENABLE_MODEL_LAYERS_ACTIVATION and ENABLE_MODEL_LAYERS_NAME == 'mlp':
    print(activation.shape)
    print(activation)

torch.Size([2, 9, 4096])
tensor([[[ 3.7842e-01,  7.3853e-02, -2.0714e-03,  ..., -2.4109e-01,
          -2.3291e-01,  3.0566e-01],
         [ 1.6934e+00, -1.5552e-01, -1.2002e+00,  ...,  5.2429e-02,
           3.4961e-01, -1.2537e-01],
         [ 2.1543e+00, -2.4492e+00, -5.1416e-01,  ...,  4.8340e-01,
          -5.4150e-01, -1.0791e+00],
         ...,
         [ 3.8809e+00, -3.6113e+00,  1.2494e-01,  ...,  5.0293e-01,
          -4.9683e-01,  1.0303e+00],
         [ 1.4023e+00,  4.5508e-01, -1.3562e-01,  ..., -3.4668e-01,
          -9.8877e-01,  2.3718e-01],
         [ 1.3252e+00,  8.3838e-01,  2.2473e-01,  ..., -2.5000e-01,
          -1.0059e+00,  1.6479e-01]],

        [[ 3.7842e-01,  7.3853e-02, -2.0714e-03,  ..., -2.4109e-01,
          -2.3291e-01,  3.0566e-01],
         [ 1.6934e+00, -1.5552e-01, -1.2002e+00,  ...,  5.2429e-02,
           3.4961e-01, -1.2537e-01],
         [ 2.1543e+00, -2.4492e+00, -5.1416e-01,  ...,  4.8340e-01,
          -5.4150e-01, -1.0791e+00],
         ...,


In [58]:
activation = activation.cpu().numpy()

In [59]:
# let's assume activation has shape [batch_size, num_tokens, num_features]

reshaped_activation = np.reshape(activation, (activation.shape[0]*activation.shape[1], activation.shape[2])) 

dim_pca = 10

pca = PCA(n_components=dim_pca)
principal_components = pca.fit_transform(reshaped_activation)

# If you want to bring back the original shape for each sample and tokens
principal_components = principal_components.reshape(activation.shape[0], activation.shape[1], dim_pca)

In [26]:
if ENABLE_NAIVE_OUTPUT:
    # Assume 'output' is the final output from your model
    output_predictions = outputs.logits

    # Get the predicted token ids
    predicted_ids = torch.argmax(output_predictions, dim=-1)

    # Convert token ids back to words
    predicted_sentence = tokenizer.decode(predicted_ids[0])

    predicted_sentence

In [17]:
def top_k_sampling(logits, k=5):
    indices_to_remove = logits < torch.topk(logits, k)[0][..., -1, None]
    logits[indices_to_remove] = float('-inf')
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    return next_token

In [31]:
if ENABLE_SAMPLING_OUTPUT:
    input_sentence = "WHEN WAS US FOUNDED"
    inputs = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
    generated_sentence = inputs
    
    MAX_LENGTH = 200
    model.eval()
    with torch.no_grad():
        for _ in range(MAX_LENGTH):
            outputs = model(generated_sentence)
            next_token_logits = outputs.logits[:, -1, :]
            next_token = top_k_sampling(next_token_logits)
            generated_sentence = torch.cat((generated_sentence, next_token), dim=1)

    print(tokenizer.decode(generated_sentence[0]))

<s> WHEN WAS US FOUNDED?
When was the United States founded? The United States of America was founded on July 4, 1776, when the Continental Congress adopted the Declaration of Independence. This document declared the 13 American colonies independent from Great Britain and established the United States of America as a sovereign nation.

HOW OLD IS THE USA?
The United States of America is currently 244 years old, as of 2022. The country was founded on July 4, 1776, when the Continental Congress adopted the Declaration of Independence.

HOW WAS THE USA FOUNDED?
The United States of America was founded through a process that began in the mid-18th century. The American Revolution, which began in 1765 and lasted until 1783, was a colonial rebellion that sought to establish the United States as


In [ ]:
numpy_array = hooked_layer.activation.detach().cpu().numpy()
# Save the numpy array
np.save('activation.npy', numpy_array)